In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import pickle

pd.set_option('display.max_rows', 500)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline  
import seaborn as sns
sns.set_style("darkgrid")

import umap
from sklearn.decomposition import TruncatedSVD as tsvd

def nearZeroVarDropAuto(df,thresh=0.99):
    vVal=df.var(axis=0).values
    cs=pd.Series(vVal).sort_values(ascending=False).cumsum()
    remove=cs[cs>cs.values[-1]*thresh].index.values
    return df.drop(df.columns[remove],axis=1)

In [3]:
%run SodaKick_download_functions.ipynb

In [4]:
df_outfield=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Outfield.hdf',key='pl')
df_team=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Team.hdf',key='pl')
df_vs=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Vs.hdf',key='pl')
df_keeper=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_keeper.hdf',key='pl')
df_fix=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_fix.hdf',key='pl')

In [7]:
all_players

#multiindex can't save on h5 with fixed format='f' needs to be table 't' but it's slower will be done on the fly
#index = pd.MultiIndex.from_arrays(df_players[['player','season','league']].T.values, names=('player','season','league'))
#df_players.drop(['player','season','league'],axis=1,inplace=True)
#df_players.index=index

player  birth_year  games  games_starts  minutes  goals  \
0         Tammy Abraham        1997   22.0          22.0   1850.0   10.0   
1      Francesco Acerbi        1988   18.0          18.0   1539.0    3.0   
2      Felix Afena-Gyan        2003    9.0           2.0    342.0    2.0   
3         Kevin Agudelo        1998    8.0           2.0    337.0    1.0   
4              Ola Aina        1996   16.0          13.0   1066.0    0.0   
...                 ...         ...    ...           ...      ...    ...   
13108       Simone Zaza        1991   33.0          23.0   2074.0   13.0   
13109     Nabil El Zhar        1986   30.0          20.0   2005.0    2.0   
13110       Luca Zidane        1998    1.0           1.0     90.0    0.0   
13111     Igor Zubeldia        1997   25.0          16.0   1589.0    1.0   
13112    David Zurutuza        1986   31.0          26.0   2236.0    0.0   

       assists  pens_made  pens_att  cards_yellow  ...  goal_kicks  \
0          3.0        0.0       0.0           6.0  ...         0.0   
1          0.0        0.0       0.0           1.0  ...         0.0   
2          0.0        0.0       0.0           4.0  ...         0.0   
3          0.0        0.0       0.0           1.0  ...         0.0   
4          0.0        0.0       0.0           4.0  ...         0.0   
...        ...        ...       ...           ...  ...         ...   
13108      2.0        0.0       0.0           9.0  ...         0.0   
13109      1.0        0.0       0.0           2.0  ...         0.0   
13110      0.0        0.0       0.0           0.0  ...         2.0   
13111      2.0        0.0       0.0           4.0  ...         0.0   
13112      2.0        0.0       0.0           2.0  ...         0.0   

       pct_goal_kicks_launched  goal_kick_length_avg  crosses_stopped  \
0                          0.0                   0.0              0.0   
1                          0.0                   0.0              0.0   
2                          0.0                   0.0              0.0   
3                          0.0                   0.0              0.0   
4                          0.0                   0.0              0.0   
...                        ...                   ...              ...   
13108                      0.0                   0.0              0.0   
13109                      0.0                   0.0              0.0   
13110                      0.0                  23.0              1.0   
13111                      0.0                   0.0              0.0   
13112                      0.0                   0.0              0.0   

       crosses_stopped_pct  def_actions_outside_pen_area  \
0                      0.0                           0.0   
1                      0.0                           0.0   
2                      0.0                           0.0   
3                      0.0                           0.0   
4                      0.0                           0.0   
...                    ...                           ...   
13108                  0.0                           0.0   
13109                  0.0                           0.0   
13110                 20.0                           0.0   
13111                  0.0                           0.0   
13112                  0.0                           0.0   

       avg_distance_def_actions  league  season  gca_og_for  
0                           0.0      SA    2122         NaN  
1                           0.0      SA    2122         NaN  
2                           0.0      SA    2122         NaN  
3                           0.0      SA    2122         NaN  
4                           0.0      SA    2122         NaN  
...                         ...     ...     ...         ...  
13108                       0.0      LL    1718         NaN  
13109                       0.0      LL    1718         NaN  
13110                      11.2      LL    1718         NaN  
13111                       0.0      LL    17

In [40]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/allplayers_weighted.pkl', 'rb') as pk:
    allpl_weighted=pickle.load(pk)
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/allplayers_weighted_tsvd.pkl', 'rb') as pk:
    allpl_weighted_ts=pickle.load(pk)

In [88]:
list(allpl_weighted[-1])

KeyError: 'team'

In [75]:
import umap
sel=allpl_weighted[-1]
sel_cut=sel
#sel_cut=sel[[x for x in allpl_weighted[3].columns if x not in ['minutes','birth_year']]][sel['keeper']==0]
mapping=umap.UMAP(metric='jaccard', n_components=2, min_dist=.5, spread=2, n_neighbors=int(np.sqrt(sel_cut.shape[0])/2), \
                                n_epochs=1000, learning_rate=0.01, \
                                verbose=False, random_state=32)

mapping.fit(sel_cut.astype(float).values)
mh=pd.DataFrame(mapping.transform(sel_cut.values), index=sel_cut.index)
mh=(mh-mh.min())/(mh.max()-mh.min())

//miniconda3/lib/python3.7/site-packages/umap/umap_.py:1530: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  "inverse_transform will be unavailable".format(self.metric)


In [49]:
from scipy.stats import spearmanr

sp=[[],[]]
for c in sel_cut.columns:
    sp[0].append(spearmanr(sel_cut[c],mh[0])[0])
    sp[1].append(spearmanr(sel_cut[c],mh[1])[0])
    
sp=pd.DataFrame(sp,columns=sel_cut.columns).T
sp.sort_values(1,ascending=False)

0         1
errors                            -0.575214  0.127734
blocked_shots_saves               -0.375373  0.117646
xa_net                            -0.008282  0.106943
own_goals                         -0.299406  0.104676
xg_net                             0.025232  0.099634
birth_year                         0.316499  0.097222
pens_conceded                     -0.477253  0.096873
cards_yellow_red                  -0.278954  0.076457
cards_red                         -0.378139  0.068177
passes_total_distance             -0.955815  0.056827
gca_passes_dead                   -0.343351  0.056021
passes_completed_medium           -0.946526  0.043018
carries                           -0.902697  0.041556
ball_recoveries                   -0.920010  0.041331
passes_free_kicks                 -0.823084  0.041147
passes_completed_long             -0.926267  0.040152
passes_oob                        -0.872031  0.040074
passes_progressive_distance       -0.930416  0.037088
carry_distance                    -0.898764  0.035551
pens_missed                       -0.128864  0.035399
pens_saved                        -0.138274  0.035036
gca_shots                         -0.355662  0.033542
minutes                           -0.786009  0.032137
free_kick_goals_against           -0.133648  0.031516
passes_offsides                   -0.780362  0.031066
passes_switches                   -0.887350  0.029888
games                             -0.870514  0.028917
through_balls                     -0.582408  0.028711
own_goals_against                 -0.129231  0.023507
passes_pressure                   -0.855054  0.020651
passes_high                       -0.897076  0.018793
psxg_net                          -0.058367  0.015343
progressive_passes                -0.801767  0.010226
passes_completed_short            -0.853138  0.006914
blocks                            -0.781579  0.004663
pens_allowed                      -0.072644  0.004191
pens_att                          -0.066965  0.000821
interceptions                     -0.780639 -0.000289
corner_kick_goals_against         -0.060909 -0.000896
cards_yellow                      -0.727124 -0.001778
clean_sheets                      -0.064431 -0.003174
passes_low                        -0.842122 -0.006402
tackles                           -0.764571 -0.008418
tackles_won                       -0.761321 -0.010332
dribble_tackles                   -0.760736 -0.010407
crosses_stopped                   -0.029189 -0.011697
def_actions_outside_pen_area      -0.030258 -0.011976
gca_fouled                        -0.288135 -0.013043
keeper                             0.029208 -0.013701
shots_free_kicks                  -0.306290 -0.014351
passes                             0.023893 -0.015060
passes_length_avg                  0.028282 -0.015547
passes_right_foot                 -0.804534 -0.016079
goal_kicks                         0.022973 -0.016318
passes_completed_launched          0.019827 -0.016589
goal_kick_length_avg               0.027245 -0.017180
passes_head                       -0.751441 -0.017368
goals_against                      0.014616 -0.017792
avg_distance_def_actions           0.025907 -0.017879
psxg                               0.021883 -0.018445
saves                              0.014523 -0.018594
shots_on_target_against            0.017049 -0.019183
npxg_per_shot                     -0.512737 -0.020437
psnpxg_per_shot_on_target_against  0.020305 -0.020796
blocked_shots                     -0.714256 -0.022429
pressure_regains                  -0.706987 -0.022860
passes_other_body                 -0.594696 -0.023701
pens_won                          -0.221981 -0.025529
goals_per_shot_on_target          -0.456743 -0.028096
assists                           -0.545587 -0.028650
passes_intercepted                -0.774484 -0.031192
pens_made                         -0.150149 -0.031923
fouls                             -0.677030 -0.035732
touches_def_pen_area              -0.791402 -0

In [84]:
fig, ax = plt.subplots(1, 1,figsize=(10,10))
#plt.axis('off')
ax.set_facecolor('white')
plt.grid(color='#aaaaaa')


plt.scatter(mh[0],mh[1], s=50, c=sel_cut['keeper'])

ax.tick_params(axis='y', which='major', labelsize=0)
ax.tick_params(axis='x', which='major', labelsize=0)


ax.set_xticks(np.linspace(0,1,6))
ax.set_xticklabels(['{:.1f}'.format(x) for x in np.linspace(0,1,6)])
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(20) 
plt.xlim((-.1,1.1))
plt.xlabel('UMAP 1', fontsize=35)



ax.set_yticks(np.linspace(0,1,6))
ax.set_yticklabels(['{:.1f}'.format(x) for x in np.linspace(0,1,6)])
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(25) 
    
plt.ylim((-.1,1.1))
plt.ylabel('UMAP 2', fontsize=30)

Text(0, 0.5, 'UMAP 2')

In [21]:
allpl_weighted[3]['x_coor']=mh[0]
allpl_weighted[3]['y_coor']=mh[1]

In [83]:
allpl_weighted[3]['keeper'].value_counts()

0.000000    4244
1.000000     154
3.266667      80
1.933333      27
2.733333      23
2.428571      18
1.666667      18
2.333333      11
1.571429       7
1.533333       7
1.857143       6
2.466667       5
4.066667       2
0.800000       1
3.000000       1
3.133333       1
Name: keeper, dtype: int64

In [78]:
mh.to_hdf('/Users/federico comitani/GitHub/sodakick/data/allpl_proj.h5',key='df')

In [22]:
allpl_weighted[3].to_csv('/Users/federico comitani/GitHub/sodakick/data/all_pl_test1.csv')

# now build them

In [ ]:
import warnings
warnings.filterwarnings('ignore')

inputs=[]
outputs=[]
lineups=[]

exit=False
for i,series in enumerate(['SA2021','SA1920','BL2021','BL1920','LU2021','LU1920','PL2021','PL1920','LL2021','LL1920']):
    
    print(series)
    
    for match in df_fixs[i]['match_report'].values:

        try:

            inp, results, lineup = clean_report(match, players_year_dict[series[2:]])
            
            if inp.shape[0]<48:
                print(match)
                exit=True
                break
            
            inputs.append(inp.astype(float).values.flatten())
            outputs.append(results.astype(float).values.flatten())
            lineups.append(lineup.reset_index())
            
        except:

            pass
        
    if exit:
        break
        
    print(len(inputs))
    
inputs=np.array(inputs)
outputs=np.array(outputs)


with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp_a_210614.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out_a_210614.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_lineup_a_210614.pkl', 'wb') as pk:
    pickle.dump(lineups,pk)

SA2021
357
SA1920
737
BL2021
1024
BL1920
1330
LU2021
1689
LU1920
1968
PL2021
2315
PL1920


In [18]:
import warnings
warnings.filterwarnings('ignore')

inputs=[]
outputs=[]
lineups=[]

exit=False
for i,series in enumerate(['SA1819','SA1718','BL1819','BL1718','LU1819','LU1718','PL1819','PL1718','LL1819','LL1718']):
    
    print(series)
    
    for match in df_fixs[i]['match_report'].values:

        try:

            inp, results, lineup = clean_report(match, players_year_dict[series[2:]])
            
            if inp.shape[0]<48:
                print(match)
                exit=True
                break

            inputs.append(inp.astype(float).values.flatten())
            outputs.append(results.astype(float).values.flatten())
            lineups.append(lineup.reset_index())
            
        except:

            pass

    print(len(inputs))
    
inputs=np.array(inputs)
outputs=np.array(outputs)


with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp_b_210614.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out_b_210614.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_lineup_b_210614.pkl', 'wb') as pk:
    pickle.dump(lineups,pk)

SA1819
357
SA1718
737
BL1819
1024
BL1718
1330
LU1819
1688
LU1718
1967
PL1819
2314
PL1718
2692
LL1819
3044
LL1718
3421


# trying and merging them

In [20]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp_a.pkl', 'rb') as pk:
    inputs=pickle.load(pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out_a.pkl', 'rb') as pk:
    outputs=pickle.load(pk)
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp_b.pkl', 'rb') as pk:
    inputs2=pickle.load(pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out_b.pkl', 'rb') as pk:
    outputs2=pickle.load(pk)

In [21]:
inputs=np.concatenate([inputs,inputs2])
outputs=np.concatenate([outputs,outputs2])

In [22]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)